In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

uniprot API로부터 PDB 리스트와 도메인 정보 정리하기

https://www.uniprot.org/uniprot/Q15910.xml # 여기를 샘플로

In [2]:
uniprot_id = 'Q15910'

In [3]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

URL = f'https://www.uniprot.org/uniprot/{uniprot_id}.xml' 
response = requests.get(URL) 
status = response.status_code 
text = response.text    # 모든 내용을 텍스트로 긁어온다.
root = ET.fromstring(response.text)    # xml 형태로 파싱한다. (여기서는 전체 트리의 시작을 root라고 지정함.)

In [4]:
# 모든 구조 보기
for x in root:
    print(x.tag, x.attrib, x.text)
    for y in x:
        print("--",y.tag, y.attrib, y.text)

{http://uniprot.org/uniprot}entry {'dataset': 'Swiss-Prot', 'created': '1998-07-15', 'modified': '2022-08-03', 'version': '210'} 
  
-- {http://uniprot.org/uniprot}accession {} Q15910
-- {http://uniprot.org/uniprot}accession {} B2RAQ1
-- {http://uniprot.org/uniprot}accession {} B3KS30
-- {http://uniprot.org/uniprot}accession {} B7Z1D6
-- {http://uniprot.org/uniprot}accession {} B7Z7L6
-- {http://uniprot.org/uniprot}accession {} Q15755
-- {http://uniprot.org/uniprot}accession {} Q75MG3
-- {http://uniprot.org/uniprot}accession {} Q92857
-- {http://uniprot.org/uniprot}accession {} Q96FI6
-- {http://uniprot.org/uniprot}name {} EZH2_HUMAN
-- {http://uniprot.org/uniprot}protein {} 
    
-- {http://uniprot.org/uniprot}gene {} 
    
-- {http://uniprot.org/uniprot}organism {} 
    
-- {http://uniprot.org/uniprot}reference {'key': '1'} 
    
-- {http://uniprot.org/uniprot}reference {'key': '2'} 
    
-- {http://uniprot.org/uniprot}reference {'key': '3'} 
    
-- {http://uniprot.org/uniprot}refer

    
-- {http://uniprot.org/uniprot}feature {'type': 'strand', 'evidence': '68'} 
    
-- {http://uniprot.org/uniprot}feature {'type': 'helix', 'evidence': '64'} 
    
-- {http://uniprot.org/uniprot}feature {'type': 'turn', 'evidence': '64'} 
    
-- {http://uniprot.org/uniprot}feature {'type': 'strand', 'evidence': '64'} 
    
-- {http://uniprot.org/uniprot}feature {'type': 'strand', 'evidence': '66'} 
    
-- {http://uniprot.org/uniprot}feature {'type': 'strand', 'evidence': '65'} 
    
-- {http://uniprot.org/uniprot}feature {'type': 'strand', 'evidence': '64'} 
    
-- {http://uniprot.org/uniprot}feature {'type': 'helix', 'evidence': '64'} 
    
-- {http://uniprot.org/uniprot}feature {'type': 'strand', 'evidence': '64'} 
    
-- {http://uniprot.org/uniprot}feature {'type': 'strand', 'evidence': '64'} 
    
-- {http://uniprot.org/uniprot}feature {'type': 'strand', 'evidence': '64'} 
    
-- {http://uniprot.org/uniprot}feature {'type': 'strand', 'evidence': '64'} 
    
-- {http://unip

In [16]:
# root에 트리 구조를 먼저 보면, entry, copyright 두 개가 있음
root.findall("./*")[:10]

[<Element '{http://uniprot.org/uniprot}entry' at 0x7fe6700d7c70>,
 <Element '{http://uniprot.org/uniprot}copyright' at 0x7fe640030270>]

In [17]:
# 여기서 모든 attribute에 동일한 요소가 붙어있음을 알 수 있다. 이걸 XML 네임스페이스(namespace), xmlns라고 함. 이를 간단하게 하기 위해 정의하자.
ns = '{http://uniprot.org/uniprot}'

In [18]:
# 여기서 entry에 있는 구성 요소를 보자
root.findall(f"./{ns}entry/*")[:10]

[<Element '{http://uniprot.org/uniprot}accession' at 0x7fe6700d7d10>,
 <Element '{http://uniprot.org/uniprot}accession' at 0x7fe6700d7d60>,
 <Element '{http://uniprot.org/uniprot}accession' at 0x7fe6700d7db0>,
 <Element '{http://uniprot.org/uniprot}accession' at 0x7fe6700d7e00>,
 <Element '{http://uniprot.org/uniprot}accession' at 0x7fe6700d7e50>,
 <Element '{http://uniprot.org/uniprot}accession' at 0x7fe6700d7ea0>,
 <Element '{http://uniprot.org/uniprot}accession' at 0x7fe6700d7ef0>,
 <Element '{http://uniprot.org/uniprot}accession' at 0x7fe6700d7810>,
 <Element '{http://uniprot.org/uniprot}accession' at 0x7fe6700d77c0>,
 <Element '{http://uniprot.org/uniprot}name' at 0x7fe6700d7770>]

In [19]:
# entry/name에 있는 값들 보기 
values = root.findall(f"./{ns}entry/{ns}name/.")
[item.text for item in values]

['EZH2_HUMAN']

In [20]:
# {http://uniprot.org/uniprot}name이라는 이름의 attrubute가 가지는 attribute 찾기 (예를 들어, attribute:name, type:synonym, value:KMT6 이런 식이다.)
[item.attrib for item in root.iter(f'{ns}name')]

[{},
 {'evidence': '53', 'type': 'primary'},
 {'type': 'synonym'},
 {'type': 'scientific'},
 {'type': 'common'},
 {},
 {},
 {},
 {},
 {},
 {}]

# 코드 초안

In [21]:
uniprot_id = 'Q15910'

In [22]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

URL = f'https://www.uniprot.org/uniprot/{uniprot_id}.xml' 
response = requests.get(URL) 
status = response.status_code 
text = response.text
root = ET.fromstring(response.text)

In [23]:
df = pd.DataFrame()
pdb_list = []
for n in root.iter('{http://uniprot.org/uniprot}dbReference'): # 여기 있는 정보를 다 가져오기
    if n.attrib['type'] == 'PDB':
        pdb_id = n.attrib['id']
        pdb_list.append(pdb_id)

df['PDB'] = pdb_list

# uniprot에 있는 domain을 df에 컬럼으로 추가하기 (Y/N을 표시하기 위한)
# 컬럼 내용으로는 서열 정보를 넣는다. 

domains_dict = dict()
for n in root.iter('{http://uniprot.org/uniprot}feature'):
    if n.attrib['type'] == 'domain':
        # 이름과 서열 첫, 끝 값을 가지고 리스트를 만들어, 이름:서열 리스트 딕셔너리 생성
        name = n.attrib['description']
        begin = int(n[0][0].attrib['position'])
        end = int(n[0][1].attrib['position'])
        seq = list(range(begin, end))    
        
        domains_dict[name] = seq    # 데이터 프레임에 컬럼 만들기 
        domain_info = name+' '+str(begin)+'-'+str(end)
        df[name] = domain_info
        print(domain_info)


CXC 503-605
SET 612-727


In [24]:
# uniprot에 등록된 PDB 정보를 읽어서 그 안에 chain 정보를 가져온다. 

#pdb_list = []
#df['PDB_CHAIN'] = None

for n in root.iter('{http://uniprot.org/uniprot}dbReference'):
    if n.attrib['type'] == 'PDB':
        for i in n.findall('{http://uniprot.org/uniprot}property'):
            if i.attrib['type'] == 'chains':
                pdb_id = n.attrib['id']
                #pdb_list.append(pdb_id)
                chain_info = i.attrib['value']
                # print(n.attrib['id'], i.attrib['value'])
                df.loc[df['PDB'] == pdb_id, 'PDB_CHAIN'] = chain_info     

In [25]:
# 읽어온 chain 정보를 split하여 begin-end로부터 서열 리스트를 만든다. 모든 chain 서열을 합친다. 
for idx in df.index:
    pdb_id = df.loc[idx, 'PDB']
    value = df.loc[idx, 'PDB_CHAIN']    # value 값을 뽑아 서열 리스트를 만든다. 
    
    seq_sum = []
    seq = value.split(',')
    for s in seq:
        seq = s.split('=')[1]
        begin = int(seq.split('-')[0])
        end = int(seq.split('-')[1])
        seq_chain = list(range(begin, end+1))
        seq_sum = seq_sum+seq_chain
        #print(pdb_id, seq, begin, end)
    
    # 위에서 생성된 도메인 딕셔너리에 서열 값과 PDB의 서열을 비교한다. PDB의 커버리지가 0.9 이하라면 실패!
    for name, domain_seq in zip(domains_dict.keys(), domains_dict.values()):
        failed_seq = []
        for seq in domain_seq:
            if seq not in seq_sum:
                failed_seq.append(seq)

        loss_ratio = len(failed_seq)/len(domain_seq)
        if loss_ratio > 0.9:
            df.loc[idx, name] = 'Y'
            # print(idx, pdb_id, name, 'Success')
        else:
            pass
            # print(idx, pdb_id, name, 'Fail')

In [26]:
df

,PDB,CXC,SET,PDB_CHAIN
0,4MI0,CXC 503-605,SET 612-727,A=520-746
1,4MI5,CXC 503-605,SET 612-727,A=521-746
2,5GSA,Y,Y,C/D=40-68
3,5H14,Y,Y,C/D=40-68
4,5H15,Y,Y,C/D=40-68
5,5H17,Y,Y,B=40-68
6,5H19,Y,Y,B=40-68
7,5H24,Y,Y,C/D=40-68
8,5H25,Y,Y,C/D=40-68
9,5HYN,CXC 503-605,SET 612-727,A/F/K/Q=1-746
